In [4]:
#30/7/24 Creates a K fold RF model for each file in a folder provided they are Fingerprints

#3/7/24 For the implementation of K-Fold RF using combined Chemopy Mutagen Data

import os
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/Endocrine/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Values'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 50):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = RandomForestClassifier(n_estimators=mtry, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = RandomForestClassifier(n_estimators=bestmetrics, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    sets += 1

For dataset NR-AhRMACCSfp.csv
for fold 1 test set mcc of 0.6869778623593988 valid set mcc of 0.5019978493710154
for fold 2 test set mcc of 0.5636291121802954 valid set mcc of 0.47439525430691426
for fold 3 test set mcc of 0.5636291121802954 valid set mcc of 0.5750962956117134
for fold 4 test set mcc of 0.6872040184525396 valid set mcc of 0.6464155840824328
for fold 5 test set mcc of 0.6902771448249767 valid set mcc of 0.6062211225767399

validation metrics of:
positives in data 148
negatives in data 187
net accuracy = 0.817910447761194
mcc = 0.6310887926727717
For dataset SR-MMPMACCSfp.csv
for fold 1 test set mcc of 0.6962811444545381 valid set mcc of 0.5545845443746958
for fold 2 test set mcc of 0.7093628331838578 valid set mcc of 0.5551075310029744
for fold 3 test set mcc of 0.5422133127250621 valid set mcc of 0.5927770575734872
for fold 4 test set mcc of 0.6554036644500243 valid set mcc of 0.5863527298559493
for fold 5 test set mcc of 0.6919364086666204 valid set mcc of 0.6103217492

for fold 4 test set mcc of 0.7027642214999339 valid set mcc of 0.2710575994548432
for fold 5 test set mcc of 0.4945491263770958 valid set mcc of 0.7007345938577394

validation metrics of:
positives in data 58
negatives in data 63
net accuracy = 0.7603305785123967
mcc = 0.525117853414576
For dataset NR-ARAvafp.csv
for fold 1 test set mcc of 0.7655570205274012 valid set mcc of 0.5344402432698848
for fold 2 test set mcc of 0.5561888604390448 valid set mcc of 0.5
for fold 3 test set mcc of 0.4003203845127178 valid set mcc of 0.6389151433378876
for fold 4 test set mcc of 0.5047443709236429 valid set mcc of 0.2573206596865631
for fold 5 test set mcc of 0.5896179161012005 valid set mcc of 0.38507702310770275

validation metrics of:
positives in data 61
negatives in data 64
net accuracy = 0.752
mcc = 0.5253335975112714
For dataset SR-AREMorganfp.csv
for fold 1 test set mcc of 0.4901251148684687 valid set mcc of 0.4873279974629188
for fold 2 test set mcc of 0.49905397538651664 valid set mcc of 

for fold 1 test set mcc of 0.38570446755682164 valid set mcc of 0.4095238095238095
for fold 2 test set mcc of 0.4741706566921629 valid set mcc of 0.5734769026410975
for fold 3 test set mcc of 0.5930673884164135 valid set mcc of 0.5909167880440763
for fold 4 test set mcc of 0.38954291055419554 valid set mcc of 0.4747252747252747
for fold 5 test set mcc of 0.41759137001809177 valid set mcc of 0.2484646732989441

validation metrics of:
positives in data 73
negatives in data 79
net accuracy = 0.6447368421052632
mcc = 0.28700381737895997
For dataset SR-ATAD5MACCSfp.csv
for fold 1 test set mcc of 0.6660869978853965 valid set mcc of 0.38138503569823695
for fold 2 test set mcc of 0.6133669020106273 valid set mcc of 0.4199942951621736
for fold 3 test set mcc of 0.5062173827984802 valid set mcc of 0.5172935265326569
for fold 4 test set mcc of 0.5541923614671027 valid set mcc of 0.38138503569823695
for fold 5 test set mcc of 0.36791434134434736 valid set mcc of 0.36

validation metrics of:
positi

for fold 5 test set mcc of 0.5331260496981621 valid set mcc of 0.4466625002869187

validation metrics of:
positives in data 61
negatives in data 64
net accuracy = 0.696
mcc = 0.4014489672896009
For dataset NR-aromataseTTorsionfp.csv
for fold 1 test set mcc of 0.5509189647176967 valid set mcc of 0.42833333333333334
for fold 2 test set mcc of 0.5056205829602306 valid set mcc of 0.47403413168626635
for fold 3 test set mcc of 0.5427786801746859 valid set mcc of 0.6959553154977629
for fold 4 test set mcc of 0.5762536864918488 valid set mcc of 0.37107422226742265
for fold 5 test set mcc of 0.532608695652174 valid set mcc of 0.4554218823344909

validation metrics of:
positives in data 58
negatives in data 63
net accuracy = 0.6942148760330579
mcc = 0.4001866494902882
For dataset SR-HSETTorsionfp.csv
for fold 1 test set mcc of 0.20261437908496732 valid set mcc of 0.39476289403185955
for fold 2 test set mcc of 0.29098960019224995 valid set mcc of 0.2792896329177484
for fold 3 test set mcc of 0.3

In [3]:
def setbalance(df):
    import pandas as pd
    import numpy as np
    #df = df.iloc[:,1:]
    toxvals = df['Toxicity_Values']

    zercount = 0
    for value in toxvals:
        if value == 0:
            zercount += 1

    poscount = len(toxvals)-zercount
    domval = 0

    if poscount > zercount:
        temp = poscount
        poscount = zercount
        poscount = temp
        domval = 1

    if poscount*4 < zercount:
        ratio = zercount / poscount
        factor = int(ratio) - 1

        loops = 0
        todel = []
        for value in toxvals:
            if loops % factor != 0 and value == domval:
                todel.append(loops)
            loops += 1

        df = df.drop(todel)
        
    return df


In [5]:
len(finalresults)

32

In [8]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Data/Model_Outputs/Fingerprint/RF_Fingerprint_Results')